## Perceptron

In [43]:
import numpy as np


class Perceptron(object):
    def __init__(self, n_inputs):
        self.weights = np.zeros(n_inputs + 1)
    
    def predict(self, inputs):
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0]
        if summation > 0:
            activation = summation
        else:
            activation = 0
        return activation

## Arquitetura da Rede Neural

In [44]:
import torch
# Permite utilizar o módulo com todas as utilidades da rede
from torch import nn
# Classe responsável por criar o dataset
from torch.utils.data import Dataset, DataLoader
# Separa aleatoriamente amostrar em um intervalo
import torch.distributions.uniform as urand


# Arquitetura
class NeuralNetwork(nn.Module):
    # Inicialização
    def __init__(self):
        super().__init__()
        self.layers = nn.Linear(1, 1)

    # Computação
    def forward(self, x):
        return self.layers(x)

## Preparação da infraestrutura de dados

In [45]:
# Dataset
class AlgebricDataset(Dataset):
    def __init__(self, f, interval, n_samples):
        X = urand.Uniform(interval[0], interval[1]).sample([n_samples]) # Amostra pares ordenados de forma uniforme 
        self.data = [(x, f(x)) for x in X]
    
    # Diz quantos dados tem no dataset
    def __len__(self):
        return len(self.data)

    # Busca dados pelo índice
    def __getitem__(self, idx):
        return self.data[idx]


# Parâmetros para utilizar na rede
line = lambda x: 2 * x + 3
interval = (-10, 10)
train_samples = 1000
test_samples = 100


# Separação do dataset de treino e de teste
train_dataset = AlgebricDataset(line, interval, train_samples)
test_dataset = AlgebricDataset(line, interval, test_samples)


# Bloco responsável por carregar os dados
train_dataloader = DataLoader(train_dataset, batch_size=train_samples, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_samples, shuffle=True)

## Hiperparâmetros de Otimização

In [46]:
# Verifica se existe uma GPU disponível, caso contrário utiliza a CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Device used: {device}')


# Instancia o modelo
model = NeuralNetwork().to(device)


# Função de perda
loss_func = nn.MSELoss()


# Gradiente descendente estocástico
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Device used: cpu


## Funções de treinamento e teste

In [47]:
# Função de treinamento
def train(model, dataloader, loss, optimizer):
    # Garante que o modelo vai ser treinado
    model.train()

    # Erro acumulado
    cumulative_loss = 0.0

    for X, y in dataloader:
        # [1, 2, 3, 4] --> [[1.0], [2.0], [3.0], [4.0]]
        X = X.unsqueeze(1).float().to(device) 
        y = y.unsqueeze(1).float().to(device)

        # Predição
        pred = model(X)

        # Erro
        loss = loss_func(pred, y)

        # Zera os gradientes acumulados
        optimizer.zero_grad()

        # Calculo dos gradientes (back propagation)
        loss.backward()

        # Anda na direção do gradiente
        optimizer.step()

        # Acumula o valor de perda
        cumulative_loss += loss.item()
    return cumulative_loss / len(dataloader)


# Função de teste
def test(model, dataloader, loss):
    # Garante que o modelo já otimizado vai ser testado
    model.eval()

    # Erro acumulado
    cumulative_loss = 0.0

    # Garante que o modelo não vai acumular os valores do gradiente
    with torch.no_grad():
        for X, y in dataloader:
        # [1, 2, 3, 4] --> [[1.0], [2.0], [3.0], [4.0]]
            X = X.unsqueeze(1).float().to(device) 
            y = y.unsqueeze(1).float().to(device)

            # Predição
            pred = model(X)

            # Erro
            loss = loss_func(pred, y)

            # Acumula o valor de do ambienteperda
            cumulative_loss += loss.item()

    return cumulative_loss / len(dataloader)

## Teste

In [48]:
epochs = 101
for t in range(epochs):
    train_loss = train(model, train_dataloader, loss_func, optimizer)
    if t % 10 == 0:
        print(f'Epoch: {t}\nTrain loss: {train_loss}\n')

test_loss = test(model, test_dataloader, loss_func)
print(f'Test loss: {test_loss}')

Epoch: 0
Train loss: 66.29534149169922

Epoch: 10
Train loss: 22.224409103393555

Epoch: 20
Train loss: 10.751258850097656

Epoch: 30
Train loss: 7.619375705718994

Epoch: 40
Train loss: 6.627624988555908

Epoch: 50
Train loss: 6.190466403961182

Epoch: 60
Train loss: 5.902285575866699

Epoch: 70
Train loss: 5.659115314483643

Epoch: 80
Train loss: 5.43410062789917

Epoch: 90
Train loss: 5.220118045806885

Epoch: 100
Train loss: 5.0150933265686035

Test loss: 4.967171669006348
